In [5]:
import tensorflow as tf

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,glob,tqdm
from ipywidgets import interact
import warnings

plt.style.use('ggplot')
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'Malgun Gothic'

In [10]:
df=pd.read_excel("./Data/training_info_0909.xlsx")
print(df.shape)
df.head()

(31211, 97)


,Unnamed: 0,timestamp,playtime,ot,start_time,end_time,schedule_day,schedule_week,uniquecode,productname,...,5day_dust,7day_dust,1day_ozone,3day_ozone,5day_ozone,7day_ozone,weight_dust&category,weight_dust&total_price,raining_1,quantity_info
0,0,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
1,1,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 여성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
2,2,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
3,3,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 여성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종
4,4,2019-01-01 06:40:00,20,2,6,60,1,1,100346,테이트 남성 셀린니트3종,...,23,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종


In [11]:
import re
def text_processing_first(df1):
    df1["p_name"] = df1.productname.str.replace("(","#")
    df1["p_name"] = df1.p_name.str.replace(")","#")
    df1["p_name"] = df1.p_name.str.upper()
    # (A무A) 인 상품 무이자 일시불로 통일 시킨다.
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("(#무#)","무이자 ",x))
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("(#일#)","일시불 ",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("LG","엘지 ",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("TV","티비",x))
    df1["p_name"]= df1["p_name"].apply(lambda x: re.sub("[A-Z0-9]","",x))    
    
  
    # 불용어(@,#,-,+,=) 문자 삭제
    df1["p_name"] = df1.p_name.apply(lambda x: re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]","",x))
    return df1

In [12]:
del df['Unnamed: 0']

In [13]:
df=text_processing_first(df)
df.head()

,timestamp,playtime,ot,start_time,end_time,schedule_day,schedule_week,uniquecode,productname,category,...,7day_dust,1day_ozone,3day_ozone,5day_ozone,7day_ozone,weight_dust&category,weight_dust&total_price,raining_1,quantity_info,p_name
0,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종
1,2019-01-01 06:00:00,20,0,6,60,1,1,100346,테이트 여성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 여성 셀린니트종
2,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종
3,2019-01-01 06:20:00,20,1,6,60,1,1,100346,테이트 여성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 여성 셀린니트종
4,2019-01-01 06:40:00,20,2,6,60,1,1,100346,테이트 남성 셀린니트3종,의류,...,29,0.002,0.017,0.02,0.002,1.144889,0.038271,0,3종,테이트 남성 셀린니트종


In [14]:
import sys
sys.version


'3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]'

In [15]:
from konlpy.tag import Okt
import re
# konlpy가 버전이 업데이트가 되면서 Okt가 twitter의 상위버전이 되어버렸다.
twitter = Okt()
def remove_len_1(data):
    word=[]
    for x in data:
        if(len(x) <=1):
            if(x.isdigit()):
                word.append(x)
            else:
                pass
        else:
            word.append(x)
    return word

In [16]:
df.loc[df["p_name"].str.contains("남성"),"sex"]=1
df.loc[df["p_name"].str.contains("여성"),"sex"]=2
df.loc[df["p_name"].str.contains("일시불"),"credit"]=1
df.loc[df["p_name"].str.contains("무이자"),"credit"]=2
df.sex.fillna(0,inplace=True)
df.credit.fillna(0,inplace=True)

In [17]:
df["p_name"] = df.p_name.apply(lambda x: twitter.morphs(x))
df["p_name"] =  df["p_name"].apply(lambda x: " ".join(remove_len_1(x)))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vec=TfidfVectorizer(analyzer='word',max_features=1500)
# vec=TfidfVectorizer(analyzer='word',max_features=1500,min_df=2,tokenizer=None)
# vec=CountVectorizer(analyzer='word',max_features=1000,min_df=2,tokenizer=None)
vec=TfidfVectorizer(analyzer='word',min_df=3,tokenizer=None)

x = vec.fit_transform(df["p_name"])

In [19]:
vec_info=pd.get_dummies(df["quantity_info"])

from sklearn.decomposition import PCA
k_pca=PCA(n_components=65,random_state=424)
info_x= k_pca.fit_transform(vec_info)

In [20]:
ttrd=np.arange(0,65).astype("str")
ttrd1 = np.full_like(ttrd,"X1_")

In [21]:
for i in range(65):
    ttrd[i] = ttrd1[i]+ttrd[i]

In [22]:
ttrd

array(['X1_0', 'X1_1', 'X1_2', 'X1_3', 'X1_4', 'X1_5', 'X1_6', 'X1_7',
       'X1_8', 'X1_9', 'X1_10', 'X1_11', 'X1_12', 'X1_13', 'X1_14',
       'X1_15', 'X1_16', 'X1_17', 'X1_18', 'X1_19', 'X1_20', 'X1_21',
       'X1_22', 'X1_23', 'X1_24', 'X1_25', 'X1_26', 'X1_27', 'X1_28',
       'X1_29', 'X1_30', 'X1_31', 'X1_32', 'X1_33', 'X1_34', 'X1_35',
       'X1_36', 'X1_37', 'X1_38', 'X1_39', 'X1_40', 'X1_41', 'X1_42',
       'X1_43', 'X1_44', 'X1_45', 'X1_46', 'X1_47', 'X1_48', 'X1_49',
       'X1_50', 'X1_51', 'X1_52', 'X1_53', 'X1_54', 'X1_55', 'X1_56',
       'X1_57', 'X1_58', 'X1_59', 'X1_60', 'X1_61', 'X1_62', 'X1_63',
       'X1_64'], dtype='<U11')

In [23]:
k_pca_info=pd.DataFrame(data=info_x,columns=ttrd)

In [24]:
df1 = pd.DataFrame(x.toarray(),columns  =vec.get_feature_names())

In [25]:
df["quantity"] = df["profit"] / df["price"]
df.loc[(df["category"]=="농수축") | (df["category"]=="이미용"),"카테고리 분류"]="대분류"
df.loc[(df["category"]=="의류") | (df["category"]=="속옷")| (df["category"]=="건강기능"),"카테고리 분류"]="중분류"
df.loc[(df["category"]=="생활용품") | (df["category"]=="주방")| (df["category"]=="가전")| (df["category"]=="잡화")| (df["category"]=="침구"),"카테고리 분류"]="소분류"
df.loc[(df["category"]=="가전") | (df["category"]=="가구"),"카테고리 분류"]="완전소분류"


In [26]:
df.columns=['timestamp', 'playtime', 'ot', 'start_time', 'end_time', 'schedule_day',
       'schedule_week', 'uniquecode', 'productname', 'category',
       'small_category', 'price', 'profit', 'quantity', 'price_grade',
       'best_schedule', 'day_count', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', '풍속(m/s)', 'Humidity',
       '이슬점온도(°C)', '전운량(10분위)', '지면온도(°C)', 'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_hum', '3day_hum', '5day_hum', '7day_hum', '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', '1day_status',
       '3day_status', '5day_status', '7day_status', '1day_discomfort',
       '3day_discomfort', '5day_discomfort', '7day_discomfort', '1day_twc',
       '3day_twc', '5day_twc', '7day_twc', '1day_windchill', '3day_windchill',
       '5day_windchill', '7day_windchill', 'Minimum_temperature',
       'Maximum_temperature', 'raining', 'year_y', 'weekend_affective',
       'total_price_grade', 'price_grade1', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '5day_U_dust', '7day_U_dust', '1day_dust', '3day_dust', '5day_dust',
       '7day_dust', '1day_ozone', '3day_ozone', '5day_ozone', '7day_ozone',
       'weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'p_name', 'sex', 'labels',"credit"]

In [27]:
train =df[['playtime', 'ot', 'end_time', 'schedule_day',
       'category', 'small_category',
       'price', 'quantity', 'price_grade',
       'best_schedule', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', 'Humidity',
       'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '1day_dust', '3day_dust', 
       '1day_ozone', '3day_ozone','weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'sex',"labels","credit"]]

In [28]:

train.columns=['playtime', 'ot', 'end_time', 'schedule_day',
       'category', 'small_category',
       'price', 'quantity', 'price_grade',
       'best_schedule', 'start_time1', 'good_sell_time',
       'schedule_month', 'schedule_dayofweek', 'temp', 'Humidity',
       'twc', 'status', 'windchill',
       'discomfort', '1day_tem', '3day_tem', '5day_tem', '7day_tem',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'Daily_cross', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time', 'U_dust', 'dust', 'ozone', 'ozone_grade',
       'dust_grade', 'U_dust_grade', '1day_U_dust', '3day_U_dust',
       '1day_dust', '3day_dust', 
       '1day_ozone', '3day_ozone','weight_dust&category', 'weight_dust&total_price', 'raining_1',
       'sex',"labels","credit"]

In [29]:
x_category =['end_time','category','small_category',
       'price_grade',
       'best_schedule', 'good_sell_time',
       'status',
       '1day_cloud',
       '3day_cloud', '5day_cloud', '7day_cloud', 'cloud_info', 
       'weekend_affective',
       'total_price_grade', 'category_total_price',
       'category_start_time1', 'rain_or_sunny', 'good_sell_brand',
       'presure_good_brand', 'brand_category_price_grade',
       'good_sell_brand_time','ozone_grade',
       'dust_grade', 'U_dust_grade','raining_1','sex','labels','credit']

In [30]:
from sklearn.preprocessing import LabelEncoder
for i in x_category:
    A=LabelEncoder()
    train[i] =A.fit_transform(train[i])

In [31]:

# 명목형 변수들은 Categorical 타입을 변환한다.
for i in x_category:
    train[i]=pd.Categorical(train[i])
target =train["quantity"]
del train["quantity"]
train=pd.get_dummies(train)
train["price"]= np.log1p(train["price"])

In [32]:
#rand = np.random.randint(low=0,high=1000,size=30)
pca = PCA(n_components=151,random_state=424)
pca1 = pca.fit_transform(df1)
pca_df=pd.DataFrame(data=pca1)

In [33]:
training = pd.concat([train,pca_df],axis=1)
training = pd.concat([training,k_pca_info],axis=1)

In [35]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=0.1,random_state=42)

In [36]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_test, label = y_test) 

In [37]:
import sys
print(sys.executable)

C:\Users\jongh\Anaconda3\python.exe


In [38]:
import tensorflow as tf

In [53]:
data_shape = X_train.shape[1]

input_data   = tf.keras.layers.Input(shape=(data_shape,))

# 0번 예측
dense1 = tf.keras.layers.Dense(512,activation='selu')(input_data)
dense_drop = tf.keras.layers.Dropout(0.5)(dense1)
dense3 = tf.keras.layers.Dense(256,activation='selu')(dense_drop)
dense4 = tf.keras.layers.Dense(256,activation='selu')(dense3)
dense5 = tf.keras.layers.Dense(256,activation='selu')(dense3)
dense6 = tf.keras.layers.Dense(256,activation='selu')(dense3)
dense7 = tf.keras.layers.Dense(64,activation='selu',kernel_regularizer='l2')(dense4)
output_data = tf.keras.layers.Dense(1,activation="relu",kernel_regularizer='l2')(dense4)

model = tf.keras.models.Model([input_data], output_data)

In [54]:
def custom_mean_squared_error(y_true, y_pred):
    return tf.math.reduce_mean(tf.abs(y_true - y_pred)/y_true)

In [55]:
from tensorflow import keras
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
loss_fn = tf.keras.losses.MeanSquaredError()
batch_size = 256

In [56]:
model.compile(optimizer=optimizer,
              loss=custom_mean_squared_error
             )

In [57]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=500,validation_data=(X_test, y_test))

Epoch 1/500
110/110 [==============================] - ETA: 0s - loss: 0.981 - ETA: 3s - loss: 0.986 - ETA: 4s - loss: 0.982 - ETA: 4s - loss: 0.977 - ETA: 5s - loss: 0.969 - ETA: 5s - loss: 0.961 - ETA: 5s - loss: 0.955 - ETA: 5s - loss: 0.945 - ETA: 5s - loss: 0.949 - ETA: 5s - loss: 0.950 - ETA: 5s - loss: 0.945 - ETA: 5s - loss: 0.945 - ETA: 5s - loss: 0.945 - ETA: 5s - loss: 0.945 - ETA: 5s - loss: 0.944 - ETA: 5s - loss: 0.942 - ETA: 5s - loss: 0.941 - ETA: 5s - loss: 0.941 - ETA: 5s - loss: 0.941 - ETA: 5s - loss: 0.939 - ETA: 5s - loss: 0.937 - ETA: 5s - loss: 0.936 - ETA: 5s - loss: 0.936 - ETA: 5s - loss: 0.936 - ETA: 5s - loss: 0.934 - ETA: 4s - loss: 0.933 - ETA: 4s - loss: 0.931 - ETA: 4s - loss: 0.930 - ETA: 4s - loss: 0.930 - ETA: 4s - loss: 0.930 - ETA: 4s - loss: 0.928 - ETA: 4s - loss: 0.927 - ETA: 4s - loss: 0.925 - ETA: 4s - loss: 0.925 - ETA: 4s - loss: 0.924 - ETA: 4s - loss: 0.922 - ETA: 4s - loss: 0.921 - ETA: 4s - loss: 0.921 - ETA: 4s - loss: 0.919 - ETA: 4s -

110/110 [==============================] - ETA: 0s - loss: 0.765 - ETA: 3s - loss: 0.767 - ETA: 4s - loss: 0.769 - ETA: 4s - loss: 0.780 - ETA: 4s - loss: 0.774 - ETA: 4s - loss: 0.772 - ETA: 4s - loss: 0.769 - ETA: 4s - loss: 0.765 - ETA: 4s - loss: 0.767 - ETA: 4s - loss: 0.761 - ETA: 4s - loss: 0.757 - ETA: 4s - loss: 0.758 - ETA: 4s - loss: 0.758 - ETA: 4s - loss: 0.757 - ETA: 4s - loss: 0.757 - ETA: 4s - loss: 0.755 - ETA: 4s - loss: 0.749 - ETA: 4s - loss: 0.750 - ETA: 4s - loss: 0.753 - ETA: 4s - loss: 0.755 - ETA: 3s - loss: 0.757 - ETA: 3s - loss: 0.757 - ETA: 3s - loss: 0.758 - ETA: 3s - loss: 0.759 - ETA: 3s - loss: 0.760 - ETA: 3s - loss: 0.761 - ETA: 3s - loss: 0.761 - ETA: 3s - loss: 0.760 - ETA: 3s - loss: 0.759 - ETA: 3s - loss: 0.760 - ETA: 3s - loss: 0.760 - ETA: 3s - loss: 0.759 - ETA: 3s - loss: 0.758 - ETA: 3s - loss: 0.756 - ETA: 3s - loss: 0.755 - ETA: 3s - loss: 0.754 - ETA: 3s - loss: 0.754 - ETA: 3s - loss: 0.753 - ETA: 2s - loss: 0.753 - ETA: 2s - loss: 0.755

110/110 [==============================] - ETA: 0s - loss: 0.681 - ETA: 2s - loss: 0.632 - ETA: 5s - loss: 0.626 - ETA: 7s - loss: 0.614 - ETA: 8s - loss: 0.608 - ETA: 7s - loss: 0.607 - ETA: 7s - loss: 0.616 - ETA: 6s - loss: 0.613 - ETA: 6s - loss: 0.609 - ETA: 6s - loss: 0.612 - ETA: 6s - loss: 0.612 - ETA: 6s - loss: 0.610 - ETA: 5s - loss: 0.606 - ETA: 5s - loss: 0.603 - ETA: 5s - loss: 0.602 - ETA: 5s - loss: 0.605 - ETA: 5s - loss: 0.607 - ETA: 4s - loss: 0.604 - ETA: 4s - loss: 0.605 - ETA: 4s - loss: 0.603 - ETA: 4s - loss: 0.602 - ETA: 4s - loss: 0.598 - ETA: 4s - loss: 0.595 - ETA: 4s - loss: 0.595 - ETA: 3s - loss: 0.593 - ETA: 3s - loss: 0.592 - ETA: 3s - loss: 0.590 - ETA: 3s - loss: 0.587 - ETA: 3s - loss: 0.586 - ETA: 3s - loss: 0.587 - ETA: 3s - loss: 0.586 - ETA: 3s - loss: 0.585 - ETA: 3s - loss: 0.585 - ETA: 3s - loss: 0.585 - ETA: 3s - loss: 0.585 - ETA: 2s - loss: 0.585 - ETA: 2s - loss: 0.585 - ETA: 2s - loss: 0.586 - ETA: 2s - loss: 0.585 - ETA: 2s - loss: 0.586

110/110 [==============================] - ETA: 0s - loss: 0.443 - ETA: 3s - loss: 0.444 - ETA: 4s - loss: 0.443 - ETA: 4s - loss: 0.436 - ETA: 5s - loss: 0.443 - ETA: 5s - loss: 0.447 - ETA: 4s - loss: 0.443 - ETA: 4s - loss: 0.441 - ETA: 4s - loss: 0.439 - ETA: 4s - loss: 0.437 - ETA: 4s - loss: 0.438 - ETA: 4s - loss: 0.439 - ETA: 4s - loss: 0.438 - ETA: 4s - loss: 0.438 - ETA: 4s - loss: 0.438 - ETA: 4s - loss: 0.437 - ETA: 4s - loss: 0.437 - ETA: 4s - loss: 0.437 - ETA: 4s - loss: 0.436 - ETA: 4s - loss: 0.436 - ETA: 4s - loss: 0.434 - ETA: 4s - loss: 0.432 - ETA: 4s - loss: 0.433 - ETA: 4s - loss: 0.432 - ETA: 3s - loss: 0.434 - ETA: 3s - loss: 0.433 - ETA: 3s - loss: 0.432 - ETA: 3s - loss: 0.431 - ETA: 3s - loss: 0.430 - ETA: 3s - loss: 0.431 - ETA: 3s - loss: 0.432 - ETA: 3s - loss: 0.433 - ETA: 3s - loss: 0.433 - ETA: 3s - loss: 0.434 - ETA: 3s - loss: 0.434 - ETA: 3s - loss: 0.434 - ETA: 3s - loss: 0.435 - ETA: 2s - loss: 0.435 - ETA: 2s - loss: 0.434 - ETA: 2s - loss: 0.434

110/110 [==============================] - ETA: 0s - loss: 0.411 - ETA: 2s - loss: 0.443 - ETA: 3s - loss: 0.430 - ETA: 3s - loss: 0.427 - ETA: 3s - loss: 0.424 - ETA: 3s - loss: 0.434 - ETA: 3s - loss: 0.434 - ETA: 4s - loss: 0.434 - ETA: 4s - loss: 0.434 - ETA: 4s - loss: 0.429 - ETA: 4s - loss: 0.427 - ETA: 4s - loss: 0.426 - ETA: 4s - loss: 0.425 - ETA: 4s - loss: 0.425 - ETA: 3s - loss: 0.425 - ETA: 3s - loss: 0.426 - ETA: 3s - loss: 0.425 - ETA: 3s - loss: 0.423 - ETA: 3s - loss: 0.422 - ETA: 3s - loss: 0.421 - ETA: 3s - loss: 0.423 - ETA: 3s - loss: 0.423 - ETA: 3s - loss: 0.423 - ETA: 3s - loss: 0.422 - ETA: 3s - loss: 0.421 - ETA: 3s - loss: 0.421 - ETA: 3s - loss: 0.421 - ETA: 3s - loss: 0.420 - ETA: 3s - loss: 0.421 - ETA: 2s - loss: 0.422 - ETA: 2s - loss: 0.422 - ETA: 2s - loss: 0.423 - ETA: 2s - loss: 0.423 - ETA: 2s - loss: 0.424 - ETA: 2s - loss: 0.423 - ETA: 2s - loss: 0.424 - ETA: 2s - loss: 0.424 - ETA: 2s - loss: 0.424 - ETA: 2s - loss: 0.424 - ETA: 2s - loss: 0.424

...

110/110 [==============================] - ETA: 0s - loss: 0.211 - ETA: 2s - loss: 0.192 - ETA: 2s - loss: 0.190 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.202 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.197 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199 - ETA: 1s - loss: 0.199

110/110 [==============================] - ETA: 0s - loss: 0.207 - ETA: 1s - loss: 0.204 - ETA: 2s - loss: 0.202 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.201 - ETA: 2s - loss: 0.199 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.199 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 1s - loss: 0.198 - ETA: 0s - loss: 0.198 - ETA: 0s - loss: 0.198 - ETA: 0s - loss: 0.198

110/110 [==============================] - ETA: 0s - loss: 0.209 - ETA: 4s - loss: 0.209 - ETA: 5s - loss: 0.206 - ETA: 6s - loss: 0.203 - ETA: 6s - loss: 0.203 - ETA: 6s - loss: 0.201 - ETA: 7s - loss: 0.201 - ETA: 7s - loss: 0.198 - ETA: 7s - loss: 0.198 - ETA: 7s - loss: 0.197 - ETA: 7s - loss: 0.198 - ETA: 7s - loss: 0.199 - ETA: 7s - loss: 0.198 - ETA: 7s - loss: 0.197 - ETA: 7s - loss: 0.195 - ETA: 7s - loss: 0.195 - ETA: 7s - loss: 0.196 - ETA: 6s - loss: 0.195 - ETA: 6s - loss: 0.194 - ETA: 5s - loss: 0.195 - ETA: 5s - loss: 0.195 - ETA: 4s - loss: 0.195 - ETA: 4s - loss: 0.196 - ETA: 4s - loss: 0.195 - ETA: 4s - loss: 0.194 - ETA: 3s - loss: 0.194 - ETA: 3s - loss: 0.194 - ETA: 3s - loss: 0.195 - ETA: 3s - loss: 0.196 - ETA: 3s - loss: 0.196 - ETA: 3s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 2s - loss: 0.197 - ETA: 1s - loss: 0.197

110/110 [==============================] - ETA: 0s - loss: 0.187 - ETA: 1s - loss: 0.185 - ETA: 2s - loss: 0.193 - ETA: 2s - loss: 0.191 - ETA: 2s - loss: 0.191 - ETA: 2s - loss: 0.192 - ETA: 2s - loss: 0.193 - ETA: 2s - loss: 0.194 - ETA: 2s - loss: 0.194 - ETA: 2s - loss: 0.194 - ETA: 2s - loss: 0.194 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.195 - ETA: 2s - loss: 0.196 - ETA: 2s - loss: 0.196 - ETA: 1s - loss: 0.196 - ETA: 1s - loss: 0.196 - ETA: 1s - loss: 0.196 - ETA: 1s - loss: 0.196 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.197 - ETA: 1s - loss: 0.196 - ETA: 0s - loss: 0.197 - ETA: 0s - loss: 0.196 - ETA: 0s - loss: 0.196

110/110 [==============================] - ETA: 0s - loss: 0.196 - ETA: 2s - loss: 0.198 - ETA: 2s - loss: 0.203 - ETA: 2s - loss: 0.202 - ETA: 2s - loss: 0.200 - ETA: 2s - loss: 0.200 - ETA: 3s - loss: 0.198 - ETA: 3s - loss: 0.199 - ETA: 3s - loss: 0.198 - ETA: 4s - loss: 0.198 - ETA: 4s - loss: 0.199 - ETA: 4s - loss: 0.198 - ETA: 4s - loss: 0.198 - ETA: 4s - loss: 0.199 - ETA: 4s - loss: 0.198 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.196 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.198 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.197 - ETA: 4s - loss: 0.196 - ETA: 4s - loss: 0.197 - ETA: 3s - loss: 0.197 - ETA: 3s - loss: 0.197 - ETA: 3s - loss: 0.197 - ETA: 3s - loss: 0.197 - ETA: 3s - loss: 0.197 - ETA: 2s - loss: 0.196